In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
from pretty_html_table import build_table
import os

jobs=[]
html_contents=[]
page_nums=['desc', 'desc&startrow=25', 'desc&startrow=50', 'desc&startrow=75']
for page_num in page_nums: #change depending on max page
    URL = 'https://jobs.teck.com/search/?q=&sortColumn=referencedate&sortDirection=' + page_num
    html_contents.append(requests.get(URL).text) 
    
for html in html_contents:
    soup = bs(html, "lxml")
    table = soup.tbody
    rows = table.find_all('tr')
    
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        jobs.append(cols) 
        df = pd.DataFrame(jobs)
df[0]=df[0].str.split('\n\n\n\n', expand=True)

df.rename(columns={0:'Title', 1: 'Location', 2: 'Posting_Date', 3:'Empty'}, inplace=True)
df.drop('Empty', axis='columns', inplace=True)
jobs = df[df['Title'].str.contains("RACE21" and "Data")].reset_index(drop=True)

# Created environmental variables to hide my username and password
username = os.environ.get('GMAIL_USERNAME')
password = os.environ.get('GMAIL_PASSWORD')

# Sending the email
msg = MIMEMultipart()

msg['From'] = os.environ.get('GMAIL_ADDRESS')   #glp.elmi@gmail.com
msg['To'] = os.environ.get('GMAIL_TO_ADDRESS')  # elmi.golpira@gmail.com
msg['subject'] = 'Teck Jobs'

output = build_table(jobs, 'orange_light')
body = "Here are the jobs you might be interested in!"

msg.attach(MIMEText(body,'plain'))
msg.attach(MIMEText(output,'html'))

server = smtplib.SMTP('smtp.gmail.com', 587)  
server.ehlo()
server.starttls()
server.login(username, password)  

server.sendmail(msg['From'], msg['To'], msg.as_string())  

# server.send_mail(output)
server.quit()

print ("Email Successfully Sent")

Email Successfully Sent
